###Разбор тестового варинта КР

In [3]:
import pandas as pd


Считываем данные: 

In [7]:
data_frame = pd.read_csv("/content/pulsar_stars.csv")
data_frame


,mip,stdip,ekip,sip,mc,stdc,ekc,sc,target
0,137.218750,45.390655,0.034985,0.134409,3.204013,17.177857,7.494588,74.003104,0
1,93.773438,41.883062,0.371402,1.291232,1.482441,12.959050,12.242911,180.158844,0
2,73.632812,35.461218,1.153607,4.161082,2.099498,13.057474,10.013591,140.894151,0
3,93.429688,39.883166,0.602060,1.518548,2.475753,14.146578,8.770640,107.970173,0
4,131.625000,44.111896,0.108905,0.239879,33.888796,70.398342,1.689460,1.031847,0
...,...,...,...,...,...,...,...,...,...
17893,121.117188,50.127279,-0.017936,-0.193900,1.566890,10.804730,13.737064,269.096109,0
17894,120.679688,40.918542,0.291465,0.627087,1.473244,13.912476,13.144044,203.416941,0
17895,138.101562,47.935314,-0.181827,-0.283746,3.270903,16.963817,6.988850,63.043499,0
17896,119.523438,44.198410,0.026662,0.352641,1.828595,12.674068,11.153939,166.582439,0


Задание 1: Сформируйте выборку для дальнейшей работы следующим образом: выберите лишь те объекты, у которых MIP  [10, 100]

In [8]:
data_condition_frame = data_frame.query('(mip >= 10 & mip <= 100)')

Укажите число строк в полученной выборке:

In [9]:
data_condition_frame.shape[0]

4218

Определите выборочное среднее для столбца MIP

In [10]:
data_condition_frame.mip.mean()

77.0531927898293

Определите минимальное значение из столбца MIP

In [11]:
data_condition_frame.mip.min()

10.0078125

Задание 2. Отсортируйте данные в столбце SIP по возрастанию, а затем разделите набор данных на обучающую и тестовую выборки с помощью train_test_split() в соотношении 80:20 c параметром random_state = 33. Используйте стратификацию по колонке отклика.

Сортируем:

In [12]:
data_frame_sip = data_condition_frame.sort_values(by = 'sip')
data_frame_sip

,mip,stdip,ekip,sip,mc,stdc,ekc,sc,target
12344,92.804688,73.323775,0.470206,-1.040048,21.126254,40.828972,1.837012,2.433000,0
3649,84.132812,74.457556,0.694280,-0.965077,98.496656,50.698540,1.170048,0.858232,0
14688,96.070312,74.560083,0.703671,-0.960082,10.634615,35.096062,3.711222,14.282259,1
795,74.632812,75.867161,0.834880,-0.838742,67.623746,81.199889,0.998825,-0.311267,0
2437,99.281250,71.956469,0.765558,-0.827640,197.234114,63.758798,-1.876388,2.431297,0
...,...,...,...,...,...,...,...,...,...
4118,12.117188,28.238001,7.525028,57.175232,123.622074,68.761628,0.047999,-0.562968,1
11783,12.507812,27.597553,7.572577,58.294501,42.612876,59.377834,1.565603,1.907165,1
12029,10.320312,26.871201,7.875742,63.149537,38.887960,58.923800,1.635320,2.225703,1
10456,18.468750,24.791612,7.595097,65.385974,69.800167,75.244969,0.894922,-0.118137,1


используем train_test_split() для поиска max(stdc)

In [14]:
from sklearn.model_selection import train_test_split
X = data_frame_sip.drop('target', axis=1)
Y = data_frame_sip['target']

x, x_test, y, y_test = train_test_split(X, Y, random_state = 33, test_size = 0.2, stratify=Y)




In [15]:
x.stdc.max()

109.6553451

Задание 3. Выполните преобразование линейной нормировки для тренировочного набора данных и примените его для тестовых данных.

In [16]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

scaled = MinMaxScaler().fit(x)
x[x.columns] = scaled.transform(x)
x_test[x_test.columns] = scaled.transform(x_test)

x.stdip.mean()

0.25572456092231716

###Используйте полученные данные и обучите алгоритмы

Обучите модель LogisticRegression() на тренировочных данных используя все параметы по умолчанию. Оцените модель на тестовых данных.

In [18]:
from sklearn.linear_model import LogisticRegression
predicted = LogisticRegression(random_state = 33).fit(x, y).predict(x_test)


Составьте матрицу ошибок. Введите значение для True Positive (TP):

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predicted)

array([[539,  18],
       [ 41, 246]])

Вычислите f1 score для тестового набора данных.

In [26]:
from sklearn.metrics import f1_score
f1_score(y_test, predicted)

0.8929219600725952

Обучите модель KNeighborsClassifier(n_neighbors=3) на тренировочных данных. Оцените на тестовых данных.

In [31]:
from sklearn.neighbors import KNeighborsClassifier
predicted = KNeighborsClassifier(n_neighbors=3).fit(x, y).predict(x_test)

Составьте матрицу ошибок. Введите значение для True Positive (TP):

In [32]:
confusion_matrix(y_test, predicted)

array([[540,  17],
       [ 29, 258]])

Вычислите f1 score для тестового набора данных.

In [34]:
f1_score(y_test, predicted)

0.918149466192171

Разбор завершен!